In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys, os

import matplotlib.animation as animation
import glob, re

import scienceplots

plt.style.use('default')
plt.style.use(['science', 'high-vis', 'grid'])

In [ ]:
def f_cost2(x, y):
    r_0 = 0, 0
    r_1 = 1, 1
    r_2 = -1, -1
    r_3 = 1.5, -1.5
    r_4 = -1.25, 1.25
    f = 0
    f += 2.0 * np.exp(-2.0 * ((x - r_1[0])**2 + (y - r_1[1])**2))
    f += 3.0 * np.exp(-2.0 * ((x - r_2[0])**2 + (y - r_2[1])**2))
    f += 4.0 * np.exp(-2.0 * ((x - r_3[0])**2 + (y - r_3[1])**2))
    f += 5.0 * np.exp(-20.0 * ((x - r_4[0])**2 + (y - r_4[1])**2))
    f += 4.0 * np.exp(-2.0 * ((x - r_0[0])**2 + (y - r_0[1])**2))
    return -f

x = np.linspace(-2, 2, 100)
y = np.linspace(-2, 2, 100)
X, Y = np.meshgrid(x, y)
Z = f_cost2(X, Y)
min , max = Z.min(), Z.max()

fig, ax = plt.subplots()
im = ax.imshow(Z, extent=[-2, 2, -2, 2], origin='lower', cmap='viridis')
fig.colorbar(im, ax=ax, label='Cost function')



In [ ]:
import numpy as np

def particle_swarm(cost_func, MaxIter, NbParticle, w, c1, c2):
    print(f'Running particle swarm optimization with {MaxIter} iterations, {NbParticle} particles, w = {w}, c1 = {c1}, c2 = {c2}')
    
    plot = True
    
    if plot:
        fig, ax = plt.subplots()
        im = ax.imshow(Z, extent=[-2, 2, -2, 2], origin='lower', cmap='viridis')
        fig.colorbar(im, ax=ax, label='Cost function')
    
    # Initialize the particles
    x = np.random.uniform(-2, 2, NbParticle)
    y = np.random.uniform(-2, 2, NbParticle)
    vx = np.zeros(NbParticle)
    vy = np.zeros(NbParticle)
    
    # Initialize the personal best positions and costs
    best_x_particle = x.copy()
    best_y_particle = y.copy()
    best_cost_particle = cost_func(x, y)
    
    # Find the global best positions and cost
    gbest_idx = np.argmin(best_cost_particle)
    best_x = x[gbest_idx]
    best_y = y[gbest_idx]
    best_cost = best_cost_particle[gbest_idx]
    
    history_best_xy = np.zeros((MaxIter, 2))
    history_best_xy[0, 0] = best_x
    history_best_xy[0, 1] = best_y
    history_best_cost = np.zeros(MaxIter)
    history_best_cost[0] = best_cost
    
    for i in range(MaxIter):
        for j in range(NbParticle):
            cost = cost_func(x[j], y[j])
            
            # Update the personal best
            if cost < best_cost_particle[j]:
                best_x_particle[j] = x[j]
                best_y_particle[j] = y[j]
                best_cost_particle[j] = cost
            
            # Update the global best
            if cost < best_cost:
                best_x = x[j]
                best_y = y[j]
                best_cost = cost
                history_best_xy[i, 0] = best_x
                history_best_xy[i, 1] = best_y
                history_best_cost[i] = best_cost
                
                # print(f'Iteration {i}, best cost = {best_cost} at x = {best_x}, y = {best_y}')
            
            # Update the velocity and position
            vx[j] = w * vx[j] + c1 * np.random.rand() * (best_x_particle[j] - x[j]) + c2 * np.random.rand() * (best_x - x[j])
            vy[j] = w * vy[j] + c1 * np.random.rand() * (best_y_particle[j] - y[j]) + c2 * np.random.rand() * (best_y - y[j])
            print(f'Iteration {i}, particle {j}, vx = {vx[j]}, vy = {vy[j]}')
            x[j] += vx[j]
            y[j] += vy[j]
            
        if plot and i % 100 == 0:
            ax.clear()
            im = ax.imshow(Z, extent=[-2, 2, -2, 2], origin='lower', cmap='viridis')
            ax.plot(best_x, best_y, 'ro')
            ax.plot(x, y, 'bo')
            print(f'Iteration {i}, best cost = {best_cost} at x = {best_x}, y = {best_y}')
            fig.savefig(f'particle_swarm_{i}.png')
    return history_best_xy, history_best_cost





In [ ]:
real_best_y = np.min(Z)
real_best_x = np.argmin(Z)
real_best_x = X.flatten()[real_best_x]
real_best_y = Y.flatten()[np.argmin(Z)]
# print(real_best_x, real_best_y)
history_best_xy, history_best_cost = particle_swarm(f_cost2, 10000, 200, 0.95, 1.5, 0.5)

best_x = history_best_xy[-1, 0]
best_y = history_best_xy[-1, 1]

fig, ax = plt.subplots()
im = ax.imshow(Z, extent=[-2, 2, -2, 2], origin='lower', cmap='viridis')
fig.colorbar(im, ax=ax, label='Cost function')

ax.plot(history_best_xy[:, 0], history_best_xy[:, 1], '-o', color='b', markersize=1)